In [23]:
import joblib
import os

# token: list
# label: pd.Series

base_dir = 'data'

train_token = joblib.load(os.path.join(base_dir, 'train_tokens.pickle'))
train_label = joblib.load(os.path.join(base_dir, 'train_label.pickle'))

test_token = joblib.load(os.path.join(base_dir, 'test.pickle'))
test_label = joblib.load(os.path.join(base_dir, 'test_label.pickle'))

len(train_token)

10000

In [24]:
from gensim.models import Word2Vec

embedding_dim = 128

word2vec_model = Word2Vec(train_token + test_token, size=embedding_dim, min_count=1, sg=1)
word2vec_model.save('model/word2vec_model')

In [25]:
import numpy as np

def build_matrix(tokens_list, word_table, embedding_dim):
    size = len(tokens_list)
    matrix = np.zeros((size, embedding_dim))
    for i, tokens in enumerate(tokens_list):
        vectorized_tokens = np.array([word_table[token] for token in tokens if token in word_table])
        matrix[i] = vectorized_tokens.mean(0)
    return np.nan_to_num(matrix)

train_x = build_matrix(train_token, word2vec_model, embedding_dim)
test_x = build_matrix(test_token, word2vec_model, embedding_dim)

In [26]:
# 진짜 간단하게 만들어보기
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KDTree, KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC

cfs = [
    GaussianNB(),
    DecisionTreeClassifier(criterion='entropy'),
    # KNeighborsClassifier(n_neighbors=2)
]

for cf in cfs:
    cf.fit(train_x, train_label)

In [27]:
from sklearn.metrics import accuracy_score

for cf in cfs:
    predictions = cf.predict(test_x)
    acc = accuracy_score(test_label, predictions)
    print(cf, acc)

GaussianNB() 0.7742946070752894
DecisionTreeClassifier(criterion='entropy') 0.6760990296397258


In [32]:
from sklearn.svm import SVC, LinearSVC

cf = SVC()
cf.fit(train_x, train_label)

SVC()

In [33]:
predictions = cf.predict(test_x)
acc = accuracy_score(test_label, predictions)
print(cf, acc)

SVC() 0.7871310291514942
